## Load Base Model

In [ ]:
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer
from pruna.algorithms.smasher_config import SmasherConfig
from pruna.smash import smash

tokenizer = AutoTokenizer.from_pretrained('facebook/opt-125m')
model = AutoModelForCausalLM.from_pretrained('facebook/opt-125m', trust_remote_code=True, torch_dtype="auto" )
model.to('cuda')
ins = tokenizer("What are we having for dinner?", return_tensors="pt", truncation=True).to('cuda')

## Smash it!

### Define Config

In [ ]:
smasher_config = SmasherConfig()
smasher_config['quantizer'] = 'bitsandbytes'
smasher_config['n_quantization_bits'] = 4
smasher_config['tokenizer_name'] = tokenizer

### Smash

In [ ]:
%%time
smashed_model = smash(
        model=model,
        data_module="Polyglot_1000",
        api_key='your-api-key',
        model_config=None,
        smasher_config=smasher_config,
        device='cuda',
    )

## Base Model Generation

In [ ]:
%%time
results = model.generate(**ins, max_length=30)

In [ ]:
output = tokenizer.decode(results[0], skip_special_tokens=True)

In [ ]:
output

## Smashed Model Generation

In [ ]:
%%time
results = smashed_model(ins.input_ids, max_length=30)

In [ ]:
output = tokenizer.decode(results[0], skip_special_tokens=True)

In [ ]:
output